In [1]:
import parse_data.preprocess as preprocess
import parse_data.loading as loading
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import scipy

In [2]:
# paths
# data_folder = '/home/tom/Documents/SWC/data' # desktop Duan Lab
# json_filename = '240913_Yansu_Jerry/2024-09-13_11-23-37_YansuFirstSolo.json' 
data_folder = r'D:\Users\Tom\OneDrive\PhD\SWC\data' # desktop home
json_filename = r'first_experiments_2409\240913\2024-09-13_11-23-37_YansuFirstSolo.json'


In [3]:
# Load JSON file into pandas df with collapsed data dictionary and adjustments based on date of recording 
df = loading.loading_pipeline(data_folder, json_filename)

Data is from period before 2024-09-13 11:23:37
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.


In [4]:
# Pre-process data 
df = preprocess.standard_preprocessing(df)

Preprocessing complete.


In [5]:
## analyse

In [32]:
def calculate_response_times(df):
    
    # response times are calculated as difference between slice onset and selected trigger activation
    # discard the first trial 
    df_slice_onset = df[df['eventDescription'] == 'slice onset'].iloc[1:]
    df_selected_trigger_activation = df[df['eventDescription'] == 'server-selected trigger activation'].iloc[1:]

    # only index as many values as there were triggers (completed trials)
    num_triggers = len(df_selected_trigger_activation['timeApplication'].values)

    # calcuate rt
    response_times = df_selected_trigger_activation['timeApplication'].values - df_slice_onset['timeApplication'].values[:num_triggers]
    response_times = pd.Series(response_times)

    # handle social
    trigger_activating_client = df_selected_trigger_activation['data.triggerClient'].values
    
    if 1 in trigger_activating_client:
        social = True
    else:
        social = False

    if social:
        trial_idxs_p1 = np.where(trigger_activating_client == 0)[0]
        trial_idxs_p2 = np.where(trigger_activating_client == 1)[0]

        response_times_p1 = response_times[trial_idxs_p1]
        response_times_p2 = response_times[trial_idxs_p2]

    # summary statistics (general)
    mean = np.mean(response_times).to_timedelta64().astype('timedelta64[ms]')
    median = np.median(response_times).astype('timedelta64[ms]')
    iqr = scipy.stats.iqr(response_times).astype('timedelta64[ms]')

    if social:
        mean_p1 = np.mean(response_times_p1).astype('timedelta64[ms]')
        mean_p2 = np.mean(response_times_p2).astype('timedelta64[ms]')

        median_p1 = np.median(response_times_p1).astype('timedelta64[ms]')
        median_p2 = np.median(response_times_p2).astype('timedelta64[ms]')

        iqr_p1 = scipy.stats.iqr(response_times_p1).astype('timedelta64[ms]')
        iqr_p2 = scipy.stats.iqr(response_times_p2).astype('timedelta64[ms]')

    if social:
        
        print(f"Mean response time: p1 - {mean_p1}, p2  - {mean_p2}.")
        print(f"Median response time: p1 - {median_p1}, p2  - {median_p2}.")
        print(f"Response time range: p1 - {iqr_p1}, p2  - {iqr_p2}.")

        print(f"Mean response time (combined): {mean}.")
        print(f"Median response time (combined): {median}.")
        print(f"Response time range (combined): {iqr}.\n")
    
    else:

        print(f"Mean response time: {mean}.")
        print(f"Median response time: {median}.")
        print(f"Response time range: {iqr}.")

    if social:
        return [response_times, response_times_p1, response_times_p2]
    else:
        return [response_times]

In [33]:
response_times = calculate_response_times(df)[0]

Mean response time: 3960 milliseconds.
Median response time: 3891 milliseconds.
Response time range: 1219 milliseconds.
